# Retention Dark days Drive

In [1]:
# importing four(BIGTABLE(Details),DARK REPORT(Dark),SEGMENTATION(df),OPEN CASES (Opcase)) tables needed for Retention daily Drive(3,18,34 days in dark) by reading all of the workbook into pandas)
# importing table 'SEGMENTATION(df)' and read as pandas.....
import pandas as pd

df = pd.read_csv('segement.csv')

In [2]:
#importing table 'BIGTABLE(Details)'and read as pandas.....
Details = pd.read_csv('Bigtable.csv',usecols = ['Full SFID','Contract Id','Type','Customer Signed By: Full Name','Customer Signed By: Phone','Customer: Secondary Phone number','Customer Warranty End Date','Price Book: Price Book Name'])

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (32) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
#importing table 'DARK REPORT'(Dark) and read as pandas.....
Dark = pd.read_csv('Dark_report.csv',usecols = ['contract_id','Days_in_Dark','LTOperiod','type','generation','ContractCreation'])

In [4]:
#importing table 'OPEN CASES (Opcase)'and read as pandas....
Opcase = pd.read_csv('Legal_FS_Open_Case.csv')

In [5]:
# Joining 3 (Dark,Details and Opcase) table together to achieved part of the required coloumns for the report where Contract id(long Contract) was used for our primary key and join1 was delcared as variable.
join1 = pd.merge(pd.merge(Dark, Details, how='left', left_on='contract_id', right_on='Full SFID'),Opcase[['Contract: Full SFID','Department','Subject']], how='left', left_on='contract_id', right_on='Contract: Full SFID')

#join1.head()

In [6]:
# Joining previous declared varible table (join1) with df table to include customer segmentation. 
# thus, a new variable was declared as Join2
join2 = pd.merge(join1, df[['Contract Id','Potential Grouping Other']], how='left', on='Contract Id')

In [7]:
# A date format was applied to both ContractCreation and Customer Warranty End Date. 
# Also normalized by removing the time stamp
join2['Customer Warranty End Date'] = pd.to_datetime(join2['Customer Warranty End Date'], format='%d/%m/%Y')

join2['ContractCreation'] = pd.to_datetime(join2['ContractCreation'], format='%Y/%m/%d')

join2['ContractCreation'] = pd.to_datetime(join2['ContractCreation']).dt.normalize()

In [8]:
#Applying  elif conditional statement to achieved customer within and outside warranty. 
#inaddition, to avoid timestamp and stir error date was converted datetime in pandas 
import datetime as date

check = pd.to_datetime('01/10/2022')

def warranty(i):
    if i >= check:
        return 'Within Warranty'
    elif i == ' ':
        return 'N/A'
    else:
        return 'Outside Warranty'
    

join2['Warranty Status'] = join2['Customer Warranty End Date'].apply(warranty)

In [9]:
# Using filters as conditional statement on Join2 coloumn='Days_in_Dark' to achieved the exact numbers of required dark days in sentence for report.
# However, an other coloumns='Dark_Days' was created on the table to identify the required dark days
import numpy as np

filters = [
    (join2.Days_in_Dark == -1),
   (join2.Days_in_Dark == -3),
   (join2.Days_in_Dark == -18),
   (join2.Days_in_Dark == -34)
]
values = ['1 DarkDay','3 DarkDays', '18 DarkDays', '34 DarkDays']

join2["Dark_Days"] = np.select(filters, values, default= join2['Days_in_Dark'])


In [10]:
# Using filters as conditional statement on Join2 coloumn='Department' to achieved the required Open cases Pool Status(CSS Pool, FS Pool, Legal Pool) in sentence for report.
# However, an other coloumns='Pool Status' was created on the table to identify the required Customer open cases Pool
filters = [
   (join2.Department == 'NG Legal'),
   (join2.Department == 'NG FS') & (join2.Subject == 'Requests / Cancel membership')
]
values = ['Legal Pool', 'FS Pool']


join2["Pool Status"] = np.select(filters, values, default= 'CSS Pool')

In [11]:
#Rearrangement of entire coloumns header to suit the report requirement [Contract Short Num	Consecutive No Activity Days	eula_free_to_use_days	Customer name	Payer number	Secondary Number	Contract Model Type	Contract Start Date	Generation	Dark Days	Warranty End Date	Warranty Status @as 1st Nov'21	Pool Status	Customer Segmentation]
# However, a new variable Join3 was created via Join2 variable. 

join3 = join2[['Contract Id','Days_in_Dark','LTOperiod','Type','Customer Signed By: Full Name','Customer Signed By: Phone',
               'Customer: Secondary Phone number','type','ContractCreation','generation','Dark_Days','Customer Warranty End Date',
               'Warranty Status','Pool Status','Potential Grouping Other','Price Book: Price Book Name']]

In [12]:
# Selection of the exact dark days('-3','-18','-34') and contract type('production') from Join3 variable table
join3 = join3[((join3['Days_in_Dark'] == -1) | (join3['Days_in_Dark'] == -3) | (join3['Days_in_Dark'] == -18) | (join3['Days_in_Dark'] == -34)) & ((join3['Price Book: Price Book Name'] != 'Direct-NG (NWO)') & (join3['Type'] == 'Production'))]


join3.columns = ['Contract Short Num','Consecutive No Activity Days','Eula_Free_to_Use_Days','Type','Customer Name',
               'Payer Number','Secondary Number','Contract Model Type','Contract Start Date','Generation','Dark Days',
               'Warranty End Date','Warranty Status','Pool Status','Customer Segmentation','Price Book: Price Book Name']


In [13]:
# dropping the coloumn="type" which is not needed in the table.
join3.drop('Type', axis = 1, inplace=True)
join3.drop('Price Book: Price Book Name', axis = 1, inplace=True)

In [14]:
# writing the final output into an excel file
join3.to_excel('Retention Call Drive.xlsx')

In [15]:
# grouped by Generation and Dark days to get summary counts
join3.groupby(['Generation','Dark Days']).count()['Contract Short Num']

Generation  Dark Days  
LEGACY      1 DarkDay      1498
            18 DarkDays     180
            3 DarkDays      799
            34 DarkDays      84
UNIFIED     1 DarkDay       109
            18 DarkDays      20
            3 DarkDays       46
            34 DarkDays      18
Name: Contract Short Num, dtype: int64